# Visualizing Swin Transformer

**by Pio Lauren T. Mendoza**

In [2]:
from PIL import Image
from torchinfo import summary

import matplotlib.pyplot as plt
import numpy as np
import requests
import torch
import torchvision.transforms as T
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"device: {device}")

%load_ext blackcellmagic

device: cpu


In [3]:
with open("image_net_1k_labels.txt") as f:
    content = f.readlines()

labels = [label.strip("""'"\n""") for label in content]

In [4]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True)
model.eval()
model.to(device);

/home/pio/workfiles/CoE197Z/visualizing-swin-transformer/venv/lib64/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
url = input()

In [7]:
img = Image.open(requests.get(url, stream=True).raw).resize((224,224)).convert('RGB')
img

MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?

In [ ]:
img_tens = transform(img).unsqueeze(0).to(device)
with torch.no_grad():
  output = model(img_tens)

In [ ]:
labels[output.max(-1).indices]

'racer, race car, racing car'

In [8]:
summary(model)

Layer (type:depth-idx)                             Param #
SwinTransformer                                    --
├─PatchEmbed: 1-1                                  --
│    └─Conv2d: 2-1                                 6,272
│    └─LayerNorm: 2-2                              256
├─Dropout: 1-2                                     --
├─Sequential: 1-3                                  --
│    └─BasicLayer: 2-3                             --
│    │    └─ModuleList: 3-1                        397,896
│    │    └─PatchMerging: 3-2                      132,096
│    └─BasicLayer: 2-4                             --
│    │    └─ModuleList: 3-3                        1,582,224
│    │    └─PatchMerging: 3-4                      526,336
│    └─BasicLayer: 2-5                             --
│    │    └─ModuleList: 3-5                        56,791,584
│    │    └─PatchMerging: 3-6                      2,101,248
│    └─BasicLayer: 2-6                             --
│    │    └─ModuleList: 3-7         

In [9]:
list(model.children())

[PatchEmbed(
   (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
   (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
 ),
 Dropout(p=0.0, inplace=False),
 Sequential(
   (0): BasicLayer(
     dim=128, input_resolution=(56, 56), depth=2
     (blocks): ModuleList(
       (0): SwinTransformerBlock(
         (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
         (attn): WindowAttention(
           (qkv): Linear(in_features=128, out_features=384, bias=True)
           (attn_drop): Dropout(p=0.0, inplace=False)
           (proj): Linear(in_features=128, out_features=128, bias=True)
           (proj_drop): Dropout(p=0.0, inplace=False)
           (softmax): Softmax(dim=-1)
         )
         (drop_path): Identity()
         (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
         (mlp): Mlp(
           (fc1): Linear(in_features=128, out_features=512, bias=True)
           (act): GELU()
           (fc2): Linear(in_features=512, out_features